In [1]:
# Import required packages/libraries:
import os
import sys
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnan, when, count, col

# Dependencies for Regression Algorithms:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import IsotonicRegression

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
25,application_1606590675617_0026,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Define sql context:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Load train and test datasets form S3:

train_df = spark.read.parquet("s3://projectmlldsongs/songs_data_split_v2/type=train/")
test_df = spark.read.parquet("s3://projectmlldsongs/songs_data_split_v2/type=test/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
##Isotonic Regression Model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Convert Rows of dataframe to Dense Vectors:
assembler = VectorAssembler(
    inputCols=['artist_familiarity','duration','loudness','key_confidence','key','end_of_fade_in',
               'time_signature_confidence','tempo','mode'],
    outputCol='features')

# Create isotonic regression object
isoreg = IsotonicRegression(labelCol='song_hotttnesss', featuresCol='features')

# Create a pipeline to sequentially perform operations on the training set:
pipeline = Pipeline(stages=[assembler, isoreg])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Perform 4-fold cross validation on the train dataset:
#paramGrid = ParamGridBuilder()\
#    .addGrid(lr.regParam, [0.1, 0.01]) \
#    .addGrid(lr.fitIntercept, [False, True])\
#    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
#    .build()

#crossval = CrossValidator(estimator=pipeline,
#                          estimatorParamMaps=paramGrid,
#                          evaluator=RegressionEvaluator(metricName="rmse",labelCol="song_hotttnesss",predictionCol="prediction"),
#                          numFolds=4)


#Train the model:
isomodel = pipeline.fit(train_df)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Make predictions on test set
predictions = isomodel.transform(test_df)

# Select example rows to display.
predictions.select("prediction", "song_hotttnesss", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="song_hotttnesss", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Print RMSE:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------------------+
|         prediction|    song_hotttnesss|            features|
+-------------------+-------------------+--------------------+
|0.27919395216835763|0.37072581745115313|[0.17980200762801...|
|0.27919395216835763|0.47267074105300666|[0.20472576641280...|
|0.27919395216835763|0.29436661170797984|[0.23282408848570...|
|0.27919395216835763| 0.2998265465430795|[0.28032429498860...|
|0.27919395216835763|0.16485407266195046|[0.31256095971592...|
+-------------------+-------------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.0578205

In [8]:
# Save trained model object in S3:

isomodel.save("s3://projectmlldsongs/saved_models_v2/isotonic_regression/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…